In [1]:
from functools import reduce
from math import sqrt
from sklearn import linear_model
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
matplotlib.style.use('ggplot')
import statsmodels.api as sm
import statsmodels.formula.api as smf
import scipy as sp

In [2]:
#(c)
def standardization(x, mean, std) : 
    return (x-mean)/std

def get_tot_diff_with_mean(x) : 
    mean_x = get_mean(x)
    return sum(map(lambda x : (x - mean_x) ** 2, x))

def get_mean(x) : 
    return (reduce(lambda value_x1, value_x2 : value_x1 + value_x2, x))/len(x)

def get_var(x) : 
    return get_tot_diff_with_mean(x)/len(x)

def get_std(x) : 
    return sqrt(get_tot_diff_with_mean(x)/(len(x) - 1))

def get_cov(x, y) : 
    mean_x = get_mean(x)
    mean_y = get_mean(y)
    sum_of_diff = sum(map(lambda value_x, value_y: (value_x-mean_x)*(value_y-mean_y), x, y))
    return sum_of_diff / (len(x) - 1)

def get_cor(x, y) : 
    return get_cov(x, y) / (get_std(x) * get_std(y))

def get_SSE(y, y_hat) :
    return sum(map(lambda value_y, value_y_hat : (value_y-value_y_hat) ** 2, y, y_hat)) / (len(y) - 2)

def get_slope(x, y) : 
    mean_x = get_mean(x)
    mean_y = get_mean(y)
    return sum(map(lambda value_x, value_y : (value_x - mean_x) * (value_y - mean_y), x, y)) / get_tot_diff_with_mean(x)

def get_intercept(x, y) : 
    return get_mean(y) - (get_slope(x, y) * get_mean(x))

def get_SST(y) : 
    return get_tot_diff_with_mean(y)

def get_R_sqr(y, y_hat) : 
    return 1 - (get_SSE(y, y_hat)/get_SST(y))

def get_std_err_of_intercept(y, y_hat, x) : 
    mean_x = get_mean(x)
    var = get_SSE(y, y_hat)
    std_err_of_intercept = sqrt(var) * sqrt((1/len(x)) + (mean_x ** 2 / get_tot_diff_with_mean(x)))    
    return std_err_of_intercept

def get_std_err_of_slope(y, y_hat, x) : 
    std_err_of_slope = sqrt(get_SSE(y, y_hat)) / sqrt(get_tot_diff_with_mean(x))
    return std_err_of_slope

def get_y_hat(slope, intercept, x) : 
    y_hat = [intercept + slope * x for x in x]
    return y_hat

def get_predicted_value(slope, intercept, given_x) : 
    predicted_value = intercept + slope * given_x
    return predicted_value

def get_std_err_of_predicted_value(SSE, x, given_x) : 
    mean_x = get_mean(x)
    std_err_of_predicted_value = SSE * sqrt(1 + (1/len(x)) + (((given_x - mean_x) ** 2) / get_tot_diff_with_mean(x)))
    return std_err_of_predicted_value

def get_confidence_interval_of_predicted_values(predicted_value, std_err_of_predicted_value, x, alpha=0.95) : 
    confidence_interval = sp.stats.t.interval(alpha=0.95, df=len(x)-2)
    print(std_err_of_predicted_value)
    print(predicted_value)
    print(confidence_interval)
    return (predicted_value + (std_err_of_predicted_value * confidence_interval[0]), predicted_value + (std_err_of_predicted_value * confidence_interval[1]))

#### 3.1
감독자 직무수행능력 데이터를 이용하여, 식 (3.12)에 주어진 적합식 $\hat{Y} = 15.3276 + 0.7803 X_1 - 0.0502 X_2$에서의 $X_1$의 계수가 3.5절에서 $X_2$의 계수에 대하여 묘사된 일련의 단순회귀식에 의하여 구해질 수 있음을 보여라.

In [3]:
performance = pd.read_csv("Supervisor_Performance.csv", sep="\t")
performance.columns = [col.strip() for col in performance.columns]

In [53]:
performance

,Y,X1,X2,X3,X4,X5,X6,W
0,43,51,30,39,61,92,45,90
1,63,64,51,54,63,73,47,118
2,71,70,68,69,76,86,48,139
3,61,63,45,47,54,84,35,110
4,81,78,56,66,71,83,47,144
5,43,55,49,44,54,49,34,99
6,58,67,42,56,66,68,35,123
7,71,75,50,55,70,66,41,130
8,72,82,72,67,71,83,31,149
9,67,61,45,47,62,80,41,108


In [54]:
performance_model = smf.ols(data=performance, formula="Y ~ X1 + X2 + X3 + X4 + X5 + X6")
result = performance_model.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      Y   R-squared:                       0.733
Model:                            OLS   Adj. R-squared:                  0.663
Method:                 Least Squares   F-statistic:                     10.50
Date:                Mon, 17 Aug 2020   Prob (F-statistic):           1.24e-05
Time:                        10:38:14   Log-Likelihood:                -97.250
No. Observations:                  30   AIC:                             208.5
Df Residuals:                      23   BIC:                             218.3
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     10.7871     11.589      0.931      0.362     -13.187      34.761
X1             0.6132      0.161      3.809      0.001       0.280       0.946
X2            -0.0731      0.136     -0.538      0.596      -0.354       0.208
X3             0.3203      0.169      1.901      0.070      -0.028       0.669
X4             0.0817      0.221      0.369      0.715      -0.376       0.540
X5             0.0384      0.147      0.261      0.796      -0.266       0.342
X6            -0.2171      0.178     -1.218      0.236      -0.586       0.152
==============================================================================
Omnibus:                        2.386   Durbin-Watson:                   1.795
Prob(Omnibus):                  0.303   Jarque-Bera (JB):                1.255
Skew:                          -0.081   Prob(JB):                        0.534
Kurtosis:                       2.011   Cond. No.                     1.34e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.34e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [4]:
# y hat과 x_1의 회귀방정식 구하기
slope = get_slope(x=performance["X1 "], y=performance["Y "])
intercept = get_intercept(x=performance["X1 "], y=performance["Y "])
print(slope)
print(intercept)
y_hat = get_y_hat(slope=slope, intercept=intercept, x=performance["X1 "])
res_err_btw_y_and_x1 = [y-predicted_value for y, predicted_value in zip(performance["Y "], y_hat)]

0.754609818719365
14.37631940662363


In [5]:
# x1과 x2의 회귀 방정식 구하기
slope = get_slope(x=performance["X1 "], y=performance["X2 "])
intercept = get_intercept(x=performance["X1 "], y=performance["X2 "])
print(slope)
print(intercept)
y_hat = get_y_hat(slope=slope, intercept=intercept, x=performance["X1 "])
res_err_btw_x1_and_x2 = [y-predicted_value for y, predicted_value in zip(performance["X2 "], y_hat)]

0.5130319769703571
18.96540366710755


In [6]:
# y에 대한 x1의 잔차와 x2에 대한 x1의 잔차의 회귀방정 식 구하기
slope = get_slope(x=res_err_btw_x1_and_x2, y=res_err_btw_y_and_x1)
intercept = get_intercept(x=res_err_btw_x1_and_x2, y=res_err_btw_y_and_x1)
print(slope)
print(intercept)
y_hat = get_y_hat(slope=slope, intercept=intercept, x=res_err_btw_x1_and_x2)
res_err_btw_yx1_and_x2x1 = [y-predicted_value for y, predicted_value in zip(res_err_btw_y_and_x1, y_hat)]

-0.050159795205249866
-1.108431528942783e-14


#### 3.3
표 3.10은 통계학 과목을 수강한 22명의 학생들에 대하여 기말시험 성적 F와 두 번의 기초 시험 성적 P1, P2를 보여준다. 

In [7]:
examination = pd.read_csv("Examination_Data.csv", sep="\t")

In [9]:
#(a) 데이터에 다음의 모형들 각각을 적합하여라
exam_model_1 = smf.ols(formula="F ~ P1", data=examination)
result_1 = exam_model_1.fit()
result_1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      F   R-squared:                       0.802
Model:                            OLS   Adj. R-squared:                  0.792
Method:                 Least Squares   F-statistic:                     81.14
Date:                Mon, 17 Aug 2020   Prob (F-statistic):           1.78e-08
Time:                        10:05:21   Log-Likelihood:                -65.930
No. Observations:                  22   AIC:                             135.9
Df Residuals:                      20   BIC:                             138.0
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    -22.3424     11.564     -1.932      0.068     -46.464       1.780
P1             1.2605      0.140      9.008      0.000       0.969       1.552
==============================================================================
Omnibus:                        1.630   Durbin-Watson:                   1.604
Prob(Omnibus):                  0.443   Jarque-Bera (JB):                1.284
Skew:                          -0.402   Prob(JB):                        0.526
Kurtosis:                       2.131   Cond. No.                         882.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [10]:
#(a) 데이터에 다음의 모형들 각각을 적합하여라
exam_model_2 = smf.ols(formula="F ~ P2", data=examination)
result_2 = exam_model_2.fit()
result_2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      F   R-squared:                       0.860
Model:                            OLS   Adj. R-squared:                  0.853
Method:                 Least Squares   F-statistic:                     122.9
Date:                Mon, 17 Aug 2020   Prob (F-statistic):           5.44e-10
Time:                        10:05:21   Log-Likelihood:                -62.128
No. Observations:                  22   AIC:                             128.3
Df Residuals:                      20   BIC:                             130.4
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -1.8535      7.562     -0.245      0.809     -17.627      13.920
P2             1.0043      0.091     11.086      0.000       0.815       1.193
==============================================================================
Omnibus:                        2.902   Durbin-Watson:                   2.064
Prob(Omnibus):                  0.234   Jarque-Bera (JB):                1.397
Skew:                          -0.558   Prob(JB):                        0.497
Kurtosis:                       3.529   Cond. No.                         693.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [11]:
#(a) 데이터에 다음의 모형들 각각을 적합하여라
exam_model_3 = smf.ols(formula="F ~ P1 + P2", data=examination)
result_3 = exam_model_3.fit()
result_3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      F   R-squared:                       0.886
Model:                            OLS   Adj. R-squared:                  0.874
Method:                 Least Squares   F-statistic:                     74.07
Date:                Mon, 17 Aug 2020   Prob (F-statistic):           1.07e-09
Time:                        10:05:21   Log-Likelihood:                -59.840
No. Observations:                  22   AIC:                             125.7
Df Residuals:                      19   BIC:                             129.0
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    -14.5005      9.236     -1.570      0.133     -33.831       4.830
P1             0.4883      0.233      2.096      0.050       0.001       0.976
P2             0.6720      0.179      3.748      0.001       0.297       1.047
==============================================================================
Omnibus:                        0.910   Durbin-Watson:                   1.778
Prob(Omnibus):                  0.634   Jarque-Bera (JB):                0.703
Skew:                          -0.410   Prob(JB):                        0.704
Kurtosis:                       2.696   Cond. No.                     1.29e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.29e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [12]:
#(b) 세 모형에 각각에 대하여 B0 = 0를 검정하여라

$t_{(20, 0.025)} = 2.086$  
$F = B_0 + B_1P_1 + \epsilon$
- $B_0$의 p-value는 0.068 < 2.08 이므로 기각할 수 없다.

$F = B_0 + B_2P_2 + \epsilon$
- $B_0$의 p-value는 0.809 < 2.08 이므로 기각할 수 없다.

$F = B_0 + B_1P_1 + B_2P_2 + \epsilon$
- $B_0$의 p-value는 0.133 < 2.08 이므로 기각할 수 없다.

In [13]:
#(c) 개별적으로 P1과 P2 중 어느 것이 F에 대하여 더 좋은 예측 변수 인가? P2의 회귀계수가 크고, p-value 또한 매우 작아 P2가 더 좋은 변수이다.

In [14]:
#(d) 첫 번째와 두 번째 기초 시험 성적이 78, 85점인 학생에 대하여 기말 시험 성적을 예측하기 위하여 어떤 모형을 사용하는 것이 좋은가?
# 각 모형에서 사용하는 변수의 수가 다르므로 수정 결정 계수를 사용 - 모형 3이 adjusted R-square가 가장 높으므로 모형 3이 적합
0.488*78+0.67*85-14.5

80.51400000000001

#### 3.4 수정된 회귀계수 $R_{a}^2$ 가 음수로 나타나는 단순 또는 다중회귀 데이터의 예를 찾거나 만들어라
수정된 회귀계수가 음수로 나타나는 경우는, 데이터 상의 컬럼 수가 로우보다 길 경우에 음수가 나올 수 있다.

#### 3.5 다음 회귀식들을 비교함으로써 단순과 다중회귀계수 사이의 관계를 살펴볼 수 있다.
회귀식들은 책 참조
단순과 다중 회귀계수 사이의 관계식 -> 따로 볼 수 있는 곳이 없을까?

#### 3.6, 3.7, 3.8, 3.9
아이패드 풀이 

#### 3.10
감독자 직무 능력 평가 데이터를 이용하여, 다음의 각 모형에 대하여 $H_0 : B_1 = B_3 = 0.5$를 검정하여라

In [33]:
performance["W"] = performance["X1"] + performance["X3"]

In [105]:
performance["X7"] = 0.5 * performance["X1"]

In [106]:
performance["X8"] = 0.5 * performance["X3"]

In [110]:
model = smf.ols(formula="Y ~ X1 + X3", data=performance)
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      Y   R-squared:                       0.708
Model:                            OLS   Adj. R-squared:                  0.686
Method:                 Least Squares   F-statistic:                     32.74
Date:                Mon, 17 Aug 2020   Prob (F-statistic):           6.06e-08
Time:                        16:02:15   Log-Likelihood:                -98.569
No. Observations:                  30   AIC:                             203.1
Df Residuals:                      27   BIC:                             207.3
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      9.8709      7.061      1.398      0.174      -4.618      24.359
X1             0.6435      0.118      5.432      0.000       0.400       0.887
X3             0.2112      0.134      1.571      0.128      -0.065       0.487
==============================================================================
Omnibus:                        6.448   Durbin-Watson:                   1.958
Prob(Omnibus):                  0.040   Jarque-Bera (JB):                1.959
Skew:                          -0.041   Prob(JB):                        0.375
Kurtosis:                       1.751   Cond. No.                         503.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [112]:
result.t_test("X3=0.5")

<class 'statsmodels.stats.contrast.ContrastResults'>
                             Test for Constraints                             
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
c0             0.2112      0.134     -2.149      0.041      -0.065       0.487

- 우선 주어진 귀무가설인 $B_1=B_3=0.5$를 파이썬으로 한 번에 검정할 수 있는 방법은 알지 못하겠다
- 하지만 2개의 등호를 쪼개서 볼 수는 있을 것 같다. $B_1=0.5$를 귀무가설로 한다던가
- 이런 경우,주어진 귀무가설을 $B_1=0.5, B_3=0.5, B_1=B_3$ 으로 쪼개서 t_test로 보고, 모두 맞을 경우 $B_1=B_3=0.5$가 맞다고 할 수 있는가?